### As an example map Free Form Text add ons to Brand list from Luxottica Manufacturer

In [6]:
# create a list containing some common add-ons
add_ons = ['polycarbonate','high-index', 'anti-reflective','polarized', 'transitions', 'blue-light protection']
print(add_ons)

['polycarbonate', 'high-index', 'anti-reflective', 'polarized', 'transitions', 'blue-light protection']


In [7]:
# now map to 33 brands under Luxottica manufacturer
brands = ['alain mikli', 'armani exchange', 'arnette', 'bolon', 'brooks brothers','burberry', 'bvlgari', 'chanel', 'coach', 'costa','dolce gabbana', 'empirio armani', 'ferrari', 'foster grant', 'georgio armani','luxottica', 'michael kors', 'miu miu', 'molsion', 'oakley','oliver peoples', 'persol', 'polo ralph lauren', 'prada', 'ralph lauren','ray-ban', 'stereoflex', 'starck', 'tiffany', 'tory borch','valentino','versace','vogue']

# Create an empty list
keywords_list = []

# Loop through brands
for brand in brands:
    # Loop through addons
    for add_on in add_ons:
        # Append combinations
        keywords_list.append([brand , brand + ' ' + add_on])
        keywords_list.append([brand , add_on + ' ' + brand])
        
# Inspect keyword list
from pprint import pprint
pprint(keywords_list)

[['alain mikli', 'alain mikli polycarbonate'],
 ['alain mikli', 'polycarbonate alain mikli'],
 ['alain mikli', 'alain mikli high-index'],
 ['alain mikli', 'high-index alain mikli'],
 ['alain mikli', 'alain mikli anti-reflective'],
 ['alain mikli', 'anti-reflective alain mikli'],
 ['alain mikli', 'alain mikli polarized'],
 ['alain mikli', 'polarized alain mikli'],
 ['alain mikli', 'alain mikli transitions'],
 ['alain mikli', 'transitions alain mikli'],
 ['alain mikli', 'alain mikli blue-light protection'],
 ['alain mikli', 'blue-light protection alain mikli'],
 ['armani exchange', 'armani exchange polycarbonate'],
 ['armani exchange', 'polycarbonate armani exchange'],
 ['armani exchange', 'armani exchange high-index'],
 ['armani exchange', 'high-index armani exchange'],
 ['armani exchange', 'armani exchange anti-reflective'],
 ['armani exchange', 'anti-reflective armani exchange'],
 ['armani exchange', 'armani exchange polarized'],
 ['armani exchange', 'polarized armani exchange'],
 ['a

In [8]:
# convert list of lists into a dataframe
import pandas as pd

# Create a DataFrame from list
keywords_df = pd.DataFrame.from_records(keywords_list)

# Print the keywords DataFrame to explore it
keywords_df.head()

,0,1
0,alain mikli,alain mikli polycarbonate
1,alain mikli,polycarbonate alain mikli
2,alain mikli,alain mikli high-index
3,alain mikli,high-index alain mikli
4,alain mikli,alain mikli anti-reflective


In [9]:
# Create column names for DataFrame
keywords_df = keywords_df.rename(columns={0: 'Brands', 1: 'Keyword_Mapping'})
keywords_df.head()

,Brands,Keyword_Mapping
0,alain mikli,alain mikli polycarbonate
1,alain mikli,polycarbonate alain mikli
2,alain mikli,alain mikli high-index
3,alain mikli,high-index alain mikli
4,alain mikli,alain mikli anti-reflective


In [10]:
# Add a manufacturer name column
keywords_df['Manufacturer_Name']='Luxottica' 
keywords_df.head()

,Brands,Keyword_Mapping,Manufacturer_Name
0,alain mikli,alain mikli polycarbonate,Luxottica
1,alain mikli,polycarbonate alain mikli,Luxottica
2,alain mikli,alain mikli high-index,Luxottica
3,alain mikli,high-index alain mikli,Luxottica
4,alain mikli,alain mikli anti-reflective,Luxottica


In [1]:
# use string of text from Luxottica website
import pandas as pd
#import text pre-processing packages
import nltk
import nltk.corpus
import re
import string
from nltk.corpus import stopwords
import nltk as nlp

text = 'Ray-Ban, one of the world’s leading lifestyle eyewear brands, and Oakley, a leader in the sport and performance category, serve as a strong base for our proprietary brand portfolio, complemented by Persol, Oliver Peoples and Alain Mikli at the high-end of the market, Costa del Mar and Arnette in the sport market, and Vogue Eyewear, Bolon, Molsion and Ossé in the affordable lifestyle market. The portfolio is rounded out by non-prescription reading glasses, including the brand Foster Grant'
tex = 'Ray-Ban, one of the world’s leading lifestyle eyewear brands, and Oakley, a leader in the sport and performance category, serve as a strong base for our proprietary brand portfolio, complemented by Persol, Oliver Peoples and Alain Mikli at the high-end of the market, Costa del Mar and Arnette in the sport market, and Vogue Eyewear, Bolon, Molsion and Ossé in the affordable lifestyle market. The portfolio is rounded out by non-prescription reading glasses, including the brand Foster Grant'

In [2]:
# Make function to lowercase text, remove punctuation, remove whitespace and remove words containing numbers
def clean_text(text):
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = text.strip()
    return text

clean = lambda x: clean_text(x)


In [3]:
# convert string into list and then list into dataframe 
List_of_Words = tex.split()
print(f'List of Words ={tex.split()}')
df = pd.DataFrame(List_of_Words,columns =['Names']) 
df.head()

List of Words =['Ray-Ban,', 'one', 'of', 'the', 'world’s', 'leading', 'lifestyle', 'eyewear', 'brands,', 'and', 'Oakley,', 'a', 'leader', 'in', 'the', 'sport', 'and', 'performance', 'category,', 'serve', 'as', 'a', 'strong', 'base', 'for', 'our', 'proprietary', 'brand', 'portfolio,', 'complemented', 'by', 'Persol,', 'Oliver', 'Peoples', 'and', 'Alain', 'Mikli', 'at', 'the', 'high-end', 'of', 'the', 'market,', 'Costa', 'del', 'Mar', 'and', 'Arnette', 'in', 'the', 'sport', 'market,', 'and', 'Vogue', 'Eyewear,', 'Bolon,', 'Molsion', 'and', 'Ossé', 'in', 'the', 'affordable', 'lifestyle', 'market.', 'The', 'portfolio', 'is', 'rounded', 'out', 'by', 'non-prescription', 'reading', 'glasses,', 'including', 'the', 'brand', 'Foster', 'Grant']


,Names
0,"Ray-Ban,"
1,one
2,of
3,the
4,world’s


In [5]:
from nltk.tokenize import word_tokenize
updated_text = pd.DataFrame(df.Names.apply(clean))
updated_text.head()

,Names
0,rayban
1,one
2,of
3,the
4,world’s
